# Automatic Parallelism

Auto-Parallel mode позволяет пользователю автоматически построить стоимостную модель и найти параллельную стратегию с более коротким временем обучения, не обращая внимания на конфигурацию стратегии. В настоящее время MindSpore поддерживает следующие две различные схемы автопараллельности:

1) Sharding Strategy Propagation Algorithm: распространение параллельной стратегии от операторов, настроенных с параллельной стратегией, к операторам, не настроенным. При распространении алгоритм пытается выбрать стратегию, которая запускает наименьшее количество тензорных перестроек связи. (L1 уровень)

2) Алгоритм поиска двойной рекурсивной стратегии: его стоимостная модель, основанная на символьных операциях, может быть свободно адаптирована к различным кластерам ускорителей и может быстро генерировать оптимальную стратегию для огромных сетей и крупномасштабного нарезки нескольких карт. (L2 уровень)

## Sharding Strategy Propagation Algorithm

Cтратегия шардирования оператора представляет собой двумерный массив, в котором указывается количество частей (шардов) по каждому измерению для каждого входного тензора этого оператора. Все части при этом равномерно нарезаны.

На основе стратегии шардирования можно вывести **распределение тензоров**, которое описывает, как тензоры распределяются по устройствам.

**Распределение тензоров**

При наличии стратегии шардирования для оператора можно определить распределение тензоров на входе и выходе этого оператора. Распределение тензоров состоит из логической матрицы устройств и отображения тензоров. Логическая матрица устройств, которая общая для входных и выходных тензоров данного оператора, представляет собой одномерный массив, описывающий организацию устройств. Отображение тензоров представляет собой двумерный массив, который показывает, как каждое измерение тензора нарезано на части, соответствующие измерению логической матрицы устройств.

Рассмотрим оператор умножения матриц (MatMul), где на вход подаются две матрицы X и W: `Y=MatMul(X,W)`. Настроим оператор со стратегией шардирования `[[2, 1], [1, 4]]`, и получим распределение тензоров и вычисления, которые выполняются на каждом устройстве (см. рисунок (b) ниже). Матрица X равномерно нарезана на 2 части по строкам, а матрица W нарезана на 4 части по столбцам. На основе стратегии шардирования логическая матрица устройств и отображение тензоров определены так, как показано на рисунке (c) ниже. Таким образом, координаты отдельных устройств описывают их позиции в логической матрице устройств. Распределение тензора на каждом устройстве определяется координатами устройства. В столбце "2" таблицы на рисунке (c) указано: устройства с 0 по 3 получают один срез, устройства с 4 по 7 получают другой срез. В столбце "4" таблицы указано: устройства 0 и 4 получают один срез, устройства 1 и 5 — другой, устройства 2 и 6 — следующий, устройства 3 и 7 — последний срез. Поэтому вычисления на каждом устройстве также определены, как показано на рисунке (d).

![ShardingPropagation](../pictures/ShardingPropagation.png)

In [ ]:
import os
import mindspore as ms
import mindspore.dataset as ds
from mindspore import nn, ops
from mindspore.communication import init
from mindspore.common.initializer import initializer

ms.set_context(mode=ms.GRAPH_MODE, save_graphs=2)
ms.set_auto_parallel_context(parallel_mode=ms.ParallelMode.AUTO_PARALLEL, search_mode="sharding_propagation")
init()
ms.set_seed(1)

class FFN(nn.Cell):
    """FeedForward Network"""
    def __init__(self):
        super().__init__()
        self.flatten = ops.Flatten()
        self.dense1 = nn.Dense(28*28, 512)
        self.relu = ops.ReLU()
        self.dense2 = nn.Dense(512, 256)
        self.dense3 = nn.Dense(256, 128)
        self.dense4 = nn.Dense(128, 64)
        self.dense5 = nn.Dense(64, 10)

    def construct(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.relu(x)
        x = self.dense3(x)
        x = self.relu(x)
        x = self.dense4(x)
        x = self.relu(x)
        x = self.dense5(x)
        return x

net = FFN()
net.dense1.matmul.shard(((2, 4), (4, 1)))
net.dense3.matmul.shard(((1, 8), (8, 1)))

def create_dataset(batch_size):
    """create dataset"""
    dataset_path = os.getenv("DATA_PATH")
    dataset = ds.MnistDataset(dataset_path)
    image_transforms = [
        ds.vision.Rescale(1.0 / 255.0, 0),
        ds.vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        ds.vision.HWC2CHW()
    ]
    label_transform = ds.transforms.TypeCast(ms.int32)
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset

data_set = create_dataset(32)
optimizer = nn.Momentum(net.trainable_params(), 1e-3, 0.1)
loss_fn = nn.CrossEntropyLoss()

def forward_fn(data, target):
    """forward propagation"""
    logits = net(data)
    loss = loss_fn(logits, target)
    return loss, logits

grad_fn = ms.value_and_grad(forward_fn, None, net.trainable_params(), has_aux=True)

@ms.jit
def train_step(inputs, targets):
    """train_step"""
    (loss_value, _), grads = grad_fn(inputs, targets)
    optimizer(grads)
    return loss_value

for epoch in range(10):
    i = 0
    for image, label in data_set:
        loss_output = train_step(image, label)
        if i % 100 == 0:
            print("epoch: %s, step: %s, loss is %s" % (epoch, i, loss_output))
        i += 1
"""
Здесь задано несколько операций.
Для операций dense1 и dense3 определены стратегии сегментирования.
Для остальных операторов стратегия определяется автоматически.
"""

### Методы создания стратегии среза для ключевых операторов

**Настройка операторов, использующих веса**

Стратегия сегментирования весов параметров очень важна, особенно для больших моделей, так как потребление памяти, вызванное весами параметров, составляет большую часть общего потребления памяти для обучения модели. Поэтому операторам, работающим с весами, обычно необходимо явно настроить стратегию шардирования. В двух приведенных ниже примерах операторы Gather и MatMul, работающие с весами, настроены с помощью стратегии шардирования, в то время как другие операторы — нет. Они соответствуют параллельному слою `VocabEmbedding` и гибридно-параллельному слою `FeedForward` в **mindformers** соответственно.

![ConfiguringOperatorsInvolvingWeights](../pictures/ShardingTechniques/ConfiguringOperatorsInvolvingWeights.png)

**Замечание:** VocabEmbedding и FeedForward - трансформеры в MindSpore

**Настройка операторов изменения размеров**

Операторы фреймворков глубокого обучения можно разделить на два типа: операторы, которые являются семантически простыми и сохраняют размерность, и операторы, изменяющие размерность входного тензора. Для операторов, сохраняющих размерность, алгоритм распространения стратегии может более легко распространять стратегию сегментирования. Однако для операторов, изменяющих размерность, явная настройка стратегии сегментирования является единственным способом лучше выразить первоначальные мысли пользователя и избежать того, чтобы алгоритм распространения стратегии получил стратегию сегментирования, которую пользователь не ожидает. В приведенном ниже примере ReduceMean и MatMul — это операторы изменения размеров, настроенные с помощью стратегии сегментирования.

![ConfiguringDimension-changingOperators](../pictures/ShardingTechniques/ConfiguringDimension-changingOperators.png)

**Настройка граничных операторов, изменяющихся при параллельном методе**

Для моделей, подобных ResNet, разные части модели имеют разную предпочтительную параллель: первая половина использует параллельные данные, а вторая половина использует параллельную модель для оптимальной итеративной производительности. Это может быть достигнуто путем настройки стратегии для граничных операторов, которые изменяются в параллельном методе. В приведенном ниже примере первый MatMul настроен со стратегией параллельной передачи данных, которая будет распространять стратегию параллельных данных вперед на первую половину модели, в то время как второй MatMul настроен со стратегией параллельной модели, которая будет распространять стратегию параллельной модели обратно на вторую половину модели.

![ConfiguringBoundaryOperatorsthatChangeinParallelMethod](../pictures/ShardingTechniques/Configuring%20BoundaryOperatorsthatChangeinParallel%20Method.png)

## Double Recursive Strategy Search Algorithm
**Замечание:** Вроде как, еще в BETA

**Замечание:** Для алгоритма поиска двойной рекурсивной стратегии не требуется никакой дополнительной настройки, за исключением контекста, приведенного выше.

Алгоритм двойного рекурсивного поиска стратегий представляет собой полностью автоматическую схему поиска стратегии на уровне оператора, где пользователю не нужно каким-либо образом настраивать модель, а алгоритм автоматически осуществляет поиск параллельных стратегий, минимизирующих затраты на коммуникацию. Основная трудность автоматического поиска стратегии на уровне оператора заключается в том, что экспоненциальный срез может повлечь за собой большое пространство для поиска, а также затраты на получение профилирующей информации, необходимой для построения модели стоимости.

Для первой задачи алгоритм поиска двойной рекурсивной стратегии суммирует свои симметричные многопорядковые характеристики путем абстрагирования обучающего кластера ИИ, так что он может эквивалентно выполнять рекурсивную дихотомию для сжатия пространства поиска из-за количества устройств; С другой стороны, алгоритм поиска с двойной рекурсивной стратегией классифицирует стоимость связи операторов, сравнивает стоимость связи внутри операторов, а также стоимость перестановки операторов и сжимает экспоненциально сложную сложность поиска до линейной путем ранжирования весов операторов.

Для второй задачи алгоритм поиска двойной рекурсивной стратегии строит символическую модель стоимости, в то время как модель стоимости традиционного подхода фокусируется на том, как точно предсказать абсолютное запаздывание различных стратегий. Стоимостная модель алгоритма двойного рекурсивного поиска стратегий сравнивает относительную стоимость различных стратегий и, таким образом, значительно экономит затраты на профилирование.

**Замечание:** Рекурсивная дихотомия - это метод оптимизации, который комбинирует принципы рекурсии и дихотомии.

Принцип работы: 
1) Функция определяется рекурсивно, используя предыдущие значения.
2) На каждой итерации интервал делится пополам.
3) Вычисляется значение функции в середине нового интервала.
4) Рекурсия продолжается до достижения заданной точности.


In [ ]:
import os
import mindspore as ms
import mindspore.dataset as ds
from mindspore import nn
from mindspore.communication import init

ms.set_context(mode=ms.GRAPH_MODE, save_graphs=2)
ms.set_auto_parallel_context(parallel_mode=ms.ParallelMode.AUTO_PARALLEL, search_mode="recursive_programming")
init()
ms.set_seed(1)

class Network(nn.Cell):
    """Network"""
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layer1 = nn.Dense(28*28, 512)
        self.layer2 = nn.Dense(512, 512)
        self.layer3 = nn.Dense(512, 1)
        self.relu = nn.ReLU()

    def construct(self, x):
        x = self.flatten(x)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.relu(x)
        logits = self.layer3(x)
        return logits

net = Network()
net.set_train()

def create_dataset(batch_size):
    """create dataset"""
    dataset_path = os.getenv("DATA_PATH")
    dataset = ds.MnistDataset(dataset_path)
    image_transforms = [
        ds.vision.Rescale(1.0 / 255.0, 0),
        ds.vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        ds.vision.HWC2CHW()
    ]
    label_transform = ds.transforms.TypeCast(ms.int32)
    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset

data_set = create_dataset(32)
optimizer = nn.Momentum(net.trainable_params(), 1e-3, 0.1)
loss_fn = nn.MAELoss()

def forward_fn(data, target):
    """forward propagation"""
    logits = net(data)
    loss = loss_fn(logits, target)
    return loss, logits

grad_fn = ms.value_and_grad(forward_fn, None, net.trainable_params(), has_aux=True)

@ms.jit
def train_step(inputs, targets):
    """train_step"""
    (loss_value, _), grads = grad_fn(inputs, targets)
    optimizer(grads)
    return loss_value

for epoch in range(10):
    i = 0
    for image, label in data_set:
        loss_output = train_step(image, label)
        if i % 100 == 0:
            print("epoch: %s, step: %s, loss is %s" % (epoch, i, loss_output))
        i += 1
